In [1]:
from pathlib import Path
import imageio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Notes
 * Metadata is a temporary dict of stuff that the Axiom code reads from files. 
 * extrinsic and intrinsic parameters are stored in a list of dicts
 * `metadata['coordinate_system']` indicates whether extrinsics are in local `'xyz'` or world `'geo'` coordinates
 
 * `lcp` is a dict lens calibration parameters, a synonym for intrinsics
 * `beta` is an numpy array that contains the values of the extrinsic dict

In [2]:
# These .py files define the objects that load calibration data and do the rectification
%run -i coastcam_funcs.py
%run -i calibration_crs.py
%run -i rectifier_crs.py


In [3]:
# List of files...three for each camera. Calibration parameters are in .json format
# These are the USGS image filename format
image_files       = ['1581508801.c1.timex.jpg','1581508801.c2.timex.jpg']
extrinsic_cal_files = ['CACO01_C1_EOBest.json','CACO01_C2_EOBest.json']
intrinsic_cal_files = ['CACO01_C1_IOBest.json','CACO01_C2_IOBest.json']

In [4]:
# Dict providing the metadata that the Axiom code infers from the USACE filename format
metadata= {'name': 'CACO-01', 'serial_number': 1, 'camera_number': 'C1', 'calibration_date': '2019-12-12', 'coordinate_system': 'geo'}
# dict providing origin and orientation of the local grid
local_origin = {'x': 410935.,'y':4655890., 'angd': 55.}

In [5]:
# read cal files and make lists of cal dicts
extrinsics_list = []
for f in extrinsic_cal_files:
    extrinsics_list.append( json2dict(f) )
intrinsics_list = []
for f in intrinsic_cal_files:
    intrinsics_list.append( json2dict(f) )

In [6]:
# check test for coordinate system
if metadata['coordinate_system'].lower() == 'xyz':
    print('Extrinsics are local coordinates')
elif metadata['coordinate_system'].lower() == 'geo':
    print('Extrinsics are in world coordinates')
else:
    print('Invalid value of coordinate_system: ',metadata['coordinate_system'])
    
print(extrinsics_list[0])
print(extrinsics_list[0]['y']-local_origin['y'])

Extrinsics are in world coordinates
{'x': 410843.97, 'y': 4655942.49, 'z': 27.3, 'a': -0.271, 't': 1.304, 'r': 0.007}
52.49000000022352


In [7]:
calibration = CameraCalibration(metadata,intrinsics_list[0],extrinsics_list[0],local_origin)
print(calibration.local_origin)
print(calibration.world_extrinsics)
print(calibration.local_extrinsics)

{'x': 410935.0, 'y': 4655890.0, 'angd': 55.0}
{'x': 410843.97, 'y': 4655942.49, 'z': 27.3, 'a': -0.271, 't': 1.304, 'r': 0.007}
{'x': -9.215372196139484, 'y': 104.67443773584442, 'z': 27.3, 'a': 0.6889310885968812, 't': 1.304, 'r': 0.007}


In [8]:
xmin = 0.
xmax = 500.
ymin = 0.
ymax = 700.
dx = 2.
dy = 2.
z =  0.

rectifier_grid = TargetGrid(
    [xmin, xmax],
    [ymin, ymax],
    dx,
    dy,
    z
)
#print(rectifier_grid.X)

rectifier = Rectifier(
    rectifier_grid
)

In [9]:
%%time
%prun rectified_image = rectifier.rectify_images(metadata, image_files, intrinsics_list, extrinsics_list, local_origin)

loop 0 calibrations:
{'NU': 2448, 'NV': 2048, 'c0U': 1150.389769, 'c0V': 1200.529802, 'fx': 3599.302724, 'fy': 3600.840045, 'd1': -0.221394, 'd2': 0.129797, 'd3': 0.0, 't1': 0.000599, 't2': -0.000393} {'x': 410843.97, 'y': 4655942.49, 'z': 27.3, 'a': -0.271, 't': 1.304, 'r': 0.007}
back from CameraCalibration
back from _find_distort_UV
(351, 251)
(2048, 2448, 3) None
back from get_pixels
back from assemble_image_weights
back from apply_weights_to_pixles
loop 1 calibrations:
{'NU': 2448, 'NV': 2048, 'c0U': 1215.323012, 'c0V': 1076.162623, 'fx': 2411.904295, 'fy': 2409.796036, 'd1': -0.219987, 'd2': 0.178703, 'd3': -0.05506, 't1': 0.000114, 't2': -0.002621} {'x': 410844.14, 'y': 4655942.41, 'z': 27.3, 'a': 0.5, 't': 1.173, 'r': 0.03}
back from CameraCalibration
back from _find_distort_UV
(351, 251)
(2048, 2448, 3) None
back from get_pixels
back from assemble_image_weights
back from apply_weights_to_pixles
 CPU times: user 2.06 s, sys: 203 ms, total: 2.27 s
Wall time: 674 ms


         10642 function calls (10400 primitive calls) in 0.667 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       17    0.268    0.016    0.268    0.016 {method 'decode' of 'ImagingDecoder' objects}
       10    0.112    0.011    0.112    0.011 {method 'astype' of 'numpy.ndarray' objects}
      512    0.066    0.000    0.066    0.000 {method 'encode' of 'ImagingEncoder' objects}
        6    0.041    0.007    0.050    0.008 interpolate.py:2503(_evaluate_linear)
       12    0.024    0.002    0.024    0.002 {method 'searchsorted' of 'numpy.ndarray' objects}
        2    0.021    0.011    0.031    0.015 rectifier_crs.py:62(_find_distort_UV)
        2    0.017    0.009    0.017    0.009 {method 'join' of 'bytes' objects}
        6    0.016    0.003    0.040    0.007 interpolate.py:2523(_find_indices)
  152/138    0.015    0.000    0.045    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      

In [11]:
rectified_image = imageio.imwrite('caco_test_image_rgi_lin.jpg',rectified_image)

Lossy conversion from float64 to uint8. Range [4.141949613382125, 255.00000000000006]. Convert image to uint8 prior to saving to suppress this warning.


In [12]:
plt.imshow( rectified_image.astype(np.int))
plt.gca().invert_yaxis()

AttributeError: 'NoneType' object has no attribute 'astype'

In [13]:
print(np.nanmax(rectified_image))

None
